Carlos Artur Guimarães

Aula Adaptada do professor Leonardo Bezerra, source: [Link](https://github.com/leobezerra/pandas-zero/tree/master/pt-br)
# Extração, transformação e carga de dados

O processo de ETL é uma parte fundamental do trabalho com dados e consiste em três etapas:

- **Extração**: a coleta de dados, potencialmente a partir de múltiplas fontes heterogêneas. Pode envolver raspagem de páginas web, acesso a interfaces de programação (APIs) ou consultas a bancos de dados.
- **Transformação**: a reorganização dos dados, envolvendo operações como união, cruzamento e agregação.
- **Carga**: a persistência do novo conjunto de dados onde se quer armazená-lo.

Este notebook foca em exemplos de métodos de transformação com o Pandas.

Para isso usaremos três dataframes artificiais em nossos exemplos: `df_a`, `df_b` e `df_c`.

## Criando dataframes a partir de dicionários

Como vamos criar dataframes customizados para nossos exemplos, precisaremos do auxílio de **dicionários**.

Um dicionário é um tipo de objeto Python que permite armazenar valores indexados por chaves, similar ao que o `DataFrame` do Pandas faz.

Usamos a notação abaixo para criar um dicionário:

```python3
nome = {
        chave1: valor1,
        chave2: valor2,
        ...
        chaveN: valorN
        }
```

Acessamos um valor em um dicionário através da sua chave, usando a notação `dicionário[chave]`.

No exemplo a seguir, o dicionário `dados_df_a` têm como chaves os nomes das séries associadas:

In [1]:
import pandas as pd

In [2]:
dados_df_a = {
            'id_indivíduo': ['1', '2', '3', '4', '5'],
            'nome': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
            'sobrenome': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']
            }

Note que cada série é representada como uma lista.

Criar um `DataFrame` a partir de um dicionário é bem simples:

In [3]:
df_a = pd.DataFrame(dados_df_a)
df_a

,id_indivíduo,nome,sobrenome
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


Seguindo o mesmo modelo, vamos criar o dataframe `df_b`:

In [4]:
dados_df_b = {
            'id_indivíduo': ['4', '5', '6', '7', '8'],
            'nome': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
            'sobrenome': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']
            }

In [5]:
df_b = pd.DataFrame(dados_df_b)
df_b

,id_indivíduo,nome,sobrenome
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [6]:
dados_df_c = {
            'id_indivíduo': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
            'id_exame': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]
            }

In [7]:
df_c = pd.DataFrame(dados_df_c)
df_c

,id_indivíduo,id_exame
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


## União de dados

Uma das operações comuns é unir observações que apresentam as mesmas características, mas estão em diferentes dataframes. 

Para isso usaremos o comando `concat` que recebe uma lista com ***n*** objetos `DataFrame` como parâmetro.

In [12]:
df_new = pd.concat([df_a, df_b])
df_new

,id_indivíduo,nome,sobrenome
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


Também seria possíve unir objetos `DataFrame` com características distintas.

No entanto, essa operação produziria um `DataFrame` com muitos dados faltando:

In [13]:
pd.concat([df_a, df_c])

,id_indivíduo,nome,sobrenome,id_exame
0,1,Alex,Anderson,NaN
1,2,Amy,Ackerman,NaN
2,3,Allen,Ali,NaN
3,4,Alice,Aoni,NaN
4,5,Ayoung,Atiches,NaN
0,1,NaN,NaN,51.0
1,2,NaN,NaN,15.0
2,3,NaN,NaN,15.0
3,4,NaN,NaN,61.0
4,5,NaN,NaN,16.0


## Cruzando dados

No exemplo anterior, vimos o resultado de unir dataframes cujas características não são idênticas.

No entanto, quanto temos pelo menos uma característica em comum entre dois dataframes, podemos **cruzar**
 esses dados, produzindo um novo dataframe que reúne toda a informação dos dataframes originais.

No exemplo, abaixo as observações do dataframe **à esquerda** (`df_a`) e do dataframe **à direita** (`df_c`) foram cruzadas, tomando como característica em comum `id_indivíduo`. 

Como você pode ver, o novo dataframe reúne as informações de ambos os dataframes usados no cruzamento dos dados:

In [14]:
pd.merge(df_a, df_c, on='id_indivíduo')

,id_indivíduo,nome,sobrenome,id_exame
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16


Em algumas situações, a mesma característica pode estar representada por diferentes nomes nos dataframes que se deseja cruzar.

Nesses casos, podemos usar os argumentos `left_on` e `right_on` para especificar, respectivamente, os nomes da característica no dataframe à esquerda e no dataframe à direita.

### Tipos de cruzamento

Uma operação de cruzamento de dados combina dados de dois dataframes que apresentem uma característica em comum.

No exemplo anterior, a característica em comum era o campo `id_indivíduo`.

Note que as observações presentes no dataframe `df_c` cujos valores para `id_indivíduo` não estão presentes no dataframe `df_a` não foram mostradas.

Se quisermos que essas observações sejam preservadas, podemos usar um **cruzamento à direita**.

In [15]:
pd.merge(df_a, df_c, on='id_indivíduo', how='right')

,id_indivíduo,nome,sobrenome,id_exame
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16
5,7,NaN,NaN,14
6,8,NaN,NaN,15
7,9,NaN,NaN,1
8,10,NaN,NaN,61
9,11,NaN,NaN,16


O resultado acima mostra tanto as observações com `id_indivíduo` presentes nos dois dataframes como o restante das observações do dataframe à direita. 

Note que as observações adicionadas pelo cruzamento à direita apresentam dados faltando.

O mesmo aconteceria se usássemos um **cruzamento à esquerda**:

In [16]:
pd.merge(df_b, df_c, on='id_indivíduo', how='left')

,id_indivíduo,nome,sobrenome,id_exame
0,4,Billy,Bonder,61.0
1,5,Brian,Black,16.0
2,6,Bran,Balwner,NaN
3,7,Bryce,Brice,14.0
4,8,Betty,Btisan,15.0


Nesse caso, a observação do dataframe `df_b` cujo `id_indivíduo` não estava presente no dataframe `df_c` foi mantida.

Em um caso mais extremo, podemos usar um **cruzamento externo**, que mantém todas as observações de ambos os dataframes:

In [17]:
pd.merge(df_b, df_c, on='id_indivíduo', how='outer')

,id_indivíduo,nome,sobrenome,id_exame
0,4,Billy,Bonder,61.0
1,5,Brian,Black,16.0
2,6,Bran,Balwner,NaN
3,7,Bryce,Brice,14.0
4,8,Betty,Btisan,15.0
5,1,NaN,NaN,51.0
6,2,NaN,NaN,15.0
7,3,NaN,NaN,15.0
8,9,NaN,NaN,1.0
9,10,NaN,NaN,61.0


## Agregando dados

As operações de união e cruzamento tem por objetivo reunir informações espalhadas em múltiplas bases em um único dataframe.

Um tipo complementar de operação é a **agregação**, que visa resumir blocos de informações através de estatísticas descritivas. 

As principais formas de agregação são obtidas por meio de pivoteamento, seja unidimensional (**grupos**) ou bidimensional (**tabelas dinâmicas**).  

### Grupos

Organizar os dados em grupos pode ser útil tanto para analisar cada grupo como para calcular estatísticas por grupo.

O primeiro passo da agregação é definir uma ou mais características usadas como fatores do agrupamento.

No exemplo abaixo, agrupamos os dados do dataset `iris`.

Este dataset é o mais baixado do repositório de aprendizado de máquina [UCI](https://archive.ics.uci.edu/ml/), listando medidas de pétalas e sépalas de três espécies de flores de íris.

Por conveniência, vamos baixá-lo da biblioteca `seaborn`:

In [18]:
import seaborn as sns
dados_íris = sns.load_dataset('iris')
dados_íris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Como podemos ver, o dataset contém largura e altura das sépalas e pétalas de 150 amostras de flor íris.

Vamos ver quantos exemplos temos por espécie:

In [19]:
dados_íris['species'].value_counts()

setosa        50
virginica     50
versicolor    50
Name: species, dtype: int64

Para agrupar este dataset por espécie, podemos usar o método `groupby()`:

In [20]:
grupos_íris = dados_íris.groupby(['species'])

Podemos, então, tratar cada um grupo como um `DataFrame` usando o método `get_group()`:

In [21]:
grupos_íris.get_group('versicolor').head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor


O agrupamento nos permite computar estatísticas sobre os grupos ao mesmo tempo ou individualmente:

#### Ao mesmo tempo

In [22]:
grupos_íris.min()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,4.3,2.3,1.0,0.1
versicolor,4.9,2.0,3.0,1.0
virginica,4.9,2.2,4.5,1.4


In [23]:
grupos_íris.max()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.8,4.4,1.9,0.6
versicolor,7.0,3.4,5.1,1.8
virginica,7.9,3.8,6.9,2.5


In [24]:
grupos_íris.mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


#### Individualmente

In [25]:
grupos_íris.get_group("versicolor").describe()

,sepal_length,sepal_width,petal_length,petal_width
count,50.000000,50.000000,50.000000,50.000000
mean,5.936000,2.770000,4.260000,1.326000
std,0.516171,0.313798,0.469911,0.197753
min,4.900000,2.000000,3.000000,1.000000
25%,5.600000,2.525000,4.000000,1.200000
50%,5.900000,2.800000,4.350000,1.300000
75%,6.300000,3.000000,4.600000,1.500000
max,7.000000,3.400000,5.100000,1.800000


In [26]:
grupo_versicolor = grupos_íris.get_group("versicolor")
grupo_versicolor.count()

sepal_length    50
sepal_width     50
petal_length    50
petal_width     50
dtype: int64

#### Agregando por múltiplas características

Um recurso poderoso do Pandas é permitir agregações a partir de múltiplas características.

Em geral, usamos esse recurso quando temos um conjunto de dados que apresentam características categóricas e númericas.

No dataset `iris`, no entanto, temos apenas uma características categórica disponível.

Vamos aproveitar essa situação e dar uma olhada em um recurso bem legal do Pandas, chamado discretização em intervalos:

In [27]:
pd.cut(dados_íris["petal_width"], bins=3)

0      (0.0976, 0.9]
1      (0.0976, 0.9]
2      (0.0976, 0.9]
3      (0.0976, 0.9]
4      (0.0976, 0.9]
           ...      
145       (1.7, 2.5]
146       (1.7, 2.5]
147       (1.7, 2.5]
148       (1.7, 2.5]
149       (1.7, 2.5]
Name: petal_width, Length: 150, dtype: category
Categories (3, interval[float64]): [(0.0976, 0.9] < (0.9, 1.7] < (1.7, 2.5]]

Entendeu o que aconteceu? 

O método `cut()` calculou os valores máximo e mínimo para a característica `petal_width` e dividiu esse intervalo em três subintervalos.

Assim, cada um dos valores originais foi substituído pelo subintervalo ao qual ele pertecene e passamos ter uma varíavel categórica 😄

Vamos substituir os dados originais pelos dados categorizados:

In [28]:
dados_íris["petal_width"] = pd.cut(dados_íris["petal_width"], bins=3)
dados_íris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,"(0.0976, 0.9]",setosa
1,4.9,3.0,1.4,"(0.0976, 0.9]",setosa
2,4.7,3.2,1.3,"(0.0976, 0.9]",setosa
3,4.6,3.1,1.5,"(0.0976, 0.9]",setosa
4,5.0,3.6,1.4,"(0.0976, 0.9]",setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,"(1.7, 2.5]",virginica
146,6.3,2.5,5.0,"(1.7, 2.5]",virginica
147,6.5,3.0,5.2,"(1.7, 2.5]",virginica
148,6.2,3.4,5.4,"(1.7, 2.5]",virginica


Um recurso adicional do Pandas para lidar com características categóricas é renomear as categorias.

Vamos renomear os subintervalos gerados.

Note que desta vez estamos alterando os dados originais diretamente usando a opção `inplace=True` (quase todos os métodos Pandas aceitam essa opção).

In [29]:
dados_íris["petal_width"].cat.rename_categories(["low", "medium", "high"], inplace=True)
dados_íris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,low,setosa
1,4.9,3.0,1.4,low,setosa
2,4.7,3.2,1.3,low,setosa
3,4.6,3.1,1.5,low,setosa
4,5.0,3.6,1.4,low,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,high,virginica
146,6.3,2.5,5.0,high,virginica
147,6.5,3.0,5.2,high,virginica
148,6.2,3.4,5.4,high,virginica


Agora que nosso dataset apresenta duas características categóricas, podemos fazer agregações por múltiplas características:

In [30]:
grupo2_íris = dados_íris.groupby(["species","petal_width"]).size()
grupo2_íris

species     petal_width
setosa      low            50
            medium          0
            high            0
versicolor  low             0
            medium         49
            high            1
virginica   low             0
            medium          5
            high           45
dtype: int64

Neste caso, em vez de produzirmos os grupos, produzimos diretamente a agregação usando o método `size()`, que conta o tamanho de cada grupo.

Pelos dados acima, podemos verificar que todas as flores de íris da espécie `setosa` presentes no dataset apresentam uma largura de pétala pequena.

Também é possível fazer uma excelente separação entre as espécies `versicolor` e `virginica`.

Note que os dados acima são uma série que apresentam um índice em múltiplos níveis (conhecido no Pandas como `MultiIndex`):

In [31]:
grupo2_íris.index

MultiIndex([(    'setosa',    'low'),
            (    'setosa', 'medium'),
            (    'setosa',   'high'),
            ('versicolor',    'low'),
            ('versicolor', 'medium'),
            ('versicolor',   'high'),
            ( 'virginica',    'low'),
            ( 'virginica', 'medium'),
            ( 'virginica',   'high')],
           names=['species', 'petal_width'])

Em meio às mensagens verbosas do Pandas, vemos que há dois níveis neste índice (`levels`), cujos nomes (`names`)  são `species` e `petal_width`.

Podemos indexar esta série de várias formas diferentes

In [32]:
grupo2_íris["virginica","high"]

45

In [33]:
grupo2_íris["virginica",]

petal_width
low        0
medium     5
high      45
dtype: int64

In [34]:
grupo2_íris[:,"high"]

species
setosa         0
versicolor     1
virginica     45
dtype: int64

Também podemos converter essa série em um `DataFrame`. 

Para isso, usamos o método `reset_index()` e informamos o nome que queremos dar à série:

In [35]:
df_íris = grupo2_íris.reset_index(name="count")
df_íris

,species,petal_width,count
0,setosa,low,50
1,setosa,medium,0
2,setosa,high,0
3,versicolor,low,0
4,versicolor,medium,49
5,versicolor,high,1
6,virginica,low,0
7,virginica,medium,5
8,virginica,high,45


### Tabelas dinâmicas

Uma outra forma de agregação disponível no Pandas é através de tabelas dinâmicas.

Neste caso, usamos o método `pivot_table()` e devemos informar as caraterísticas para o agrupamento a nível de linhas (`index`) e de colunas (`columns`).

Também podemos informar um método de agregação usando a opção `aggfunc`, que por padrão calcula a média:

In [36]:
pt_íris = dados_íris.pivot_table(index="species", columns="petal_width", aggfunc="size")
pt_íris

petal_width,low,medium,high
species,,,
setosa,50,0,0
versicolor,0,49,1
virginica,0,5,45


Note que a tabela dinâmica tenta gerar todas as combinações possíveis entre os valores das característica de linha e de coluna.

Como nosso dataset não apresenta observações da espécie `setosa` com largura de pétala `medium` ou `high`, esses valores são marcados como faltando/inválidos.

O método `pivot_table()` fornece a opção `fill_value`, que nos permite escolher como preencher esses casos:

In [37]:
pt_íris = dados_íris.pivot_table(index="species", columns="petal_width", aggfunc="size", fill_value=0)
pt_íris

petal_width,low,medium,high
species,,,
setosa,50,0,0
versicolor,0,49,1
virginica,0,5,45


O método `pivot_table()` produz um objeto do tipo `DataFrame`.

Assim, a indexação funciona da maneira como já conhecemos:

In [38]:
pt_íris.loc["versicolor"]

petal_width
low        0
medium    49
high       1
Name: versicolor, dtype: int64

In [39]:
pt_íris.loc["versicolor","low"]

0

In [40]:
pt_íris.loc[:,"low"]

species
setosa        50
versicolor     0
virginica      0
Name: low, dtype: int64